# 0. Imports

In [4]:
import sys
import os

current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)
sys.path.append(os.path.join(parent_dir, 'scripts'))

import data_acquisition
import Dataframe


# 1. Collecting Data

In [5]:
data_acquisition.collect_data()

fetching data for 2016_2017 ..


Fetching game data for 2016-2017 Season: 100%|█| 1335/1335 [00:28<00:00, 47.54it


saving data for 2016_2017 ..
fetching data for 2017_2018 ..


Fetching game data for 2017-2018 Season: 100%|█| 1376/1376 [00:30<00:00, 45.39it


saving data for 2017_2018 ..
fetching data for 2018_2019 ..


...

saving data for 2018_2019 ..
fetching data for 2019_2020 ..


Fetching game data for 2019-2020 Season: 100%|█| 1376/1376 [00:35<00:00, 38.63it


saving data for 2019_2020 ..
fetching data for 2020_2021 ..


Fetching game data for 2020-2021 Season: 100%|█| 1376/1376 [00:26<00:00, 51.65it


saving data for 2020_2021 ..


# 2. Cleaning Data

In [6]:
base_path = '../data/raw/'
df = Dataframe.load_all_seasons(base_path)
transformed_df = Dataframe.transformEventData(df)

Seasons:  33%|████████████                        | 2/6 [00:24<00:53, 13.32s/it]
Loading .ipynb_checkpoints: 0it [00:00, ?it/s]
                                              
Seasons: 100%|████████████████████████████████████| 6/6 [01:09<00:00, 11.51s/it]
